In [1]:
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import math
import statistics
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd
from random import sample
import warnings
from termcolor import colored
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gc
pd.set_option('mode.chained_assignment', None)
pd.set_option('max_columns', None)

# Analyzing Speed Data: INRIX + Traffic Counter Data
## Comprehensive Analysis of Traffic Speed Data for the State of Virginia

### Counter Speed Data

In [2]:
# Read the common file only once
is_routes_tag_attr_df = pd.read_csv('test_tms_speed\\IS_routes_tag_attr.csv')

# Extract unique and non-NA LINKIDs
Study_area_LINKID = is_routes_tag_attr_df[['LINKID']].drop_duplicates().dropna()

# Get all segment attribute
Seg_attr = is_routes_tag_attr_df.copy()

# Read the LINKID_continuous file
LINKID_continuous = pd.read_csv('test_tms_speed\\speed_counter_data\\SearchLinkIDs.csv')


#### Speed Counter Data transformation and cleaning

In [3]:
import glob
import os
import pandas as pd ## Import pandas library

rootdir = os.getcwd() ## Get current working directory
list_output = [] ## Create empty list called list_output
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith("_datapull.csv"): ## Check if file name ends with "_datapull.csv"
            print(os.path.join(subdir, file)) ## Print file path
            list_output.append(os.path.join(subdir, file)) ## Add file path to list_output

result = pd.DataFrame() ## Create empty pandas DataFrame called result
for file in list_output:
    for chunk in pd.read_csv(file, names=["LINKID", "DateTime", "Dir", "CounterNumber", "Lane", "MPH_0_15", "MPH_15_25", "MPH_25_30", "MPH_30_35", "MPH_35_40", "MPH_40_45", "MPH_45_50", "MPH_50_55", "MPH_55_60", "MPH_60_65", "MPH_65_70", "MPH_70_75", "MPH_75_80", "MPH_80_85", "MPH_85_UP", "TotalVol","VolQual"],dtype={'CounterNumber':'object','Lane':'object'}, chunksize=5000000): 
        ## Read file as pandas DataFrame with specified column names and data types, and chunksize of 5000000 rows
        result = pd.concat([result, chunk], ignore_index=True) ## Concatenate the chunks into result DataFrame

result = result.drop_duplicates() ## Drop duplicate rows from the result DataFrame

D:\Daily_progress\Speed_LOS_VDF\LOS_module\test_tms_speed\speed_counter_data\speeddata_0214_datapull.csv


In [4]:
# Convert the 'DateTime' column to datetime format, inferring the format automatically.
result['DateTime_formatted'] = pd.to_datetime(result['DateTime'], infer_datetime_format=True)

# Extract the year from the 'DateTime_formatted' and store it in a new 'Year' column.
result['Year'] = result['DateTime_formatted'].dt.year

# Extract the month from the 'DateTime_formatted' and store it in a new 'Mon' (Month) column.
result['Mon'] = result['DateTime_formatted'].dt.month

# Extract the day of the month from the 'DateTime_formatted' and store it in a new 'Day' column.
result['Day'] = result['DateTime_formatted'].dt.day

# Extract the hour from the 'DateTime_formatted' and store it in a new 'Hour' column.
result['Hour'] = result['DateTime_formatted'].dt.hour

# Extract the day of the week as an integer (0 for Monday, 6 for Sunday) and store it in a new 'Wkday' column.
result['Wkday'] = result['DateTime_formatted'].dt.dayofweek

# Create a 'Date' column by combining year, month, and day, and converting it to an integer format (YYYYMMDD).
result['Date'] = (result['DateTime_formatted'].dt.year * 10000 + result['DateTime_formatted'].dt.month * 100 + result['DateTime_formatted'].dt.day).astype(int)


In [5]:
# Indicate that there is no data coverage from the continuous count table.
print("No data coverage from continuous count table")

# Calculate the set difference using Pandas built-in methods for better efficiency.
difference = set(LINKID_continuous['LinkID'].unique()) - set(result['LINKID'].unique())

# Sort the set for better readability and then print it.
print(f"LinkIDs in LINKID_continuous but not in result: {sorted(difference)}")


No data coverage from continuous count table
LinkIDs in LINKID_continuous but not in result: [50154, 120025, 150156, 782797, 782834, 782838]


In [6]:
# Print the message to indicate the purpose of the output
print("LINKIDs that do not have all coverage from 2017 - 2021")

# Perform the groupby and filtering operation
grouped_result = result.groupby(["LINKID", "Year"]).size().reset_index(name='TotalVol')
filtered_result = grouped_result.groupby('LINKID').filter(lambda x: len(x) < 5)

# Extract unique LINKIDs and convert them to a set for efficient removal
list_coverage_missing = set(filtered_result['LINKID'].unique())

# Remove the specific LINKIDs 190073 and 199750
list_coverage_missing -= {190073, 199750}

# Convert back to list and sort for readability
list_coverage_missing = sorted(list_coverage_missing)

# Print the list of remaining LINKIDs with incomplete coverage
print(f"Incomplete coverage for LINKIDs: {list_coverage_missing}")


LINKIDs that do not have all coverage from 2017 - 2021
Incomplete coverage for LINKIDs: [10005, 10148, 10181, 10185, 40042, 40056, 40099, 40412, 40489, 40551, 40648, 40765, 50119, 50149, 120013, 140013, 140015, 140026, 140031, 140048, 150036, 150037, 150038, 160105, 180005, 190029, 199774, 781288, 781963, 791023]


In [8]:
# remove the LINKIDs that does not have full year  coverage
result = result[~result.LINKID.isin(list_coverage_missing)]

In [9]:
# Calculate unique LINKIDs only once for efficiency
unique_study_area_LINKID = set(Study_area_LINKID['LINKID'].unique())
unique_result_LINKID = set(result['LINKID'].unique())

# Get unique LINKIDs from Study_area_LINKID that are not in result
unique_in_request_list = unique_study_area_LINKID - unique_result_LINKID

# Create a DataFrame of unique LINKIDs from Study_area_LINKID
summary_df = pd.DataFrame({'Requested_LINKID': list(unique_study_area_LINKID)})

# Merge with DataFrame of missing LINKIDs using a left join
summary_df = pd.merge(
    summary_df, 
    pd.DataFrame({'missing_data_LINKID': list(unique_in_request_list)}),  
    how='left', 
    left_on=['Requested_LINKID'], 
    right_on=['missing_data_LINKID']
)

# Print summary information
print(f"Total Requested LINKIDs: {summary_df['Requested_LINKID'].nunique()}")
print(f"Total available LINKIDs from Speed Counter: {summary_df['missing_data_LINKID'].isna().sum()}")

# Show the summary DataFrame
summary_df

Total Requested LINKIDs: 1423
Total available LINKIDs from Speed Counter: 453


,Requested_LINKID,missing_data_LINKID
0,90112.0,90112.0
1,90114.0,90114.0
2,90116.0,90116.0
3,90118.0,90118.0
4,90120.0,90120.0
...,...,...
1418,90051.0,90051.0
1419,980188.0,980188.0
1420,90106.0,NaN
1421,90108.0,90108.0


In [10]:
# Save the unique LINKIDs from the result DataFrame to a CSV file
pd.DataFrame({'Speed_counter_Available_LINKID': list(result['LINKID'].unique())}).to_csv('output\\Available_speed_counter_LINKID.csv')


In [11]:
# Save the unique missing_data_LINKID values from the summary_df DataFrame to a CSV file
pd.DataFrame({'missing_data_LINKID': list(summary_df['missing_data_LINKID'].dropna().unique())}).to_csv('output\\missing_speed_counter_LINKID.csv')


In [12]:
print("Speed counter data structure")
result.head()

Speed counter data structure


,LINKID,DateTime,Dir,CounterNumber,Lane,MPH_0_15,MPH_15_25,MPH_25_30,MPH_30_35,MPH_35_40,MPH_40_45,MPH_45_50,MPH_50_55,MPH_55_60,MPH_60_65,MPH_65_70,MPH_70_75,MPH_75_80,MPH_80_85,MPH_85_UP,TotalVol,VolQual,DateTime_formatted,Year,Mon,Day,Hour,Wkday,Date
0,40301,01/01/2017 00:00,N,1,3,0,0,0,0,3,2,17,39,15,9,2,0,0,0,0,87,4,2017-01-01 00:00:00,2017,1,1,0,6,20170101
1,40301,01/01/2017 00:15,N,1,3,0,0,0,0,0,7,17,69,36,4,2,0,0,0,0,135,4,2017-01-01 00:15:00,2017,1,1,0,6,20170101
2,40301,01/01/2017 00:30,N,1,3,0,1,0,0,0,15,46,70,29,15,2,0,0,0,0,178,4,2017-01-01 00:30:00,2017,1,1,0,6,20170101
3,40301,01/01/2017 00:45,N,1,3,0,0,0,0,4,19,44,50,31,4,4,0,0,0,0,156,4,2017-01-01 00:45:00,2017,1,1,0,6,20170101
4,40301,01/01/2017 01:00,N,1,3,0,0,0,0,0,9,38,46,28,4,1,0,0,0,0,126,4,2017-01-01 01:00:00,2017,1,1,1,6,20170101


#### Calculate Average speed from the volume and speed brackets

In [13]:
# calculate the hourly aggrgate of vol/spd summary
grouped = result.groupby(["LINKID","Dir","Date","Hour"])
df_vol_spd = grouped.agg({'MPH_0_15': 'sum', 
                     'MPH_15_25': 'sum',
                     'MPH_25_30': 'sum', 
                     'MPH_30_35': 'sum',
                     'MPH_35_40': 'sum',
                     'MPH_40_45': 'sum', 
                     'MPH_45_50': 'sum', 
                     'MPH_50_55': 'sum',
                     'MPH_55_60': 'sum',
                     'MPH_60_65': 'sum',
                     'MPH_65_70': 'sum',
                     'MPH_70_75': 'sum',
                     'MPH_75_80': 'sum',
                     'MPH_80_85': 'sum',
                     'MPH_85_UP': 'sum',
                     'TotalVol': 'sum',
                     'VolQual': 'mean',
                     'Year': 'mean',
                     'Mon': 'mean',
                     'Day': 'mean',
                     'Wkday': 'mean'}).reset_index()

In [14]:
# Delete individual variables to free up memory
del chunk, grouped, result


In [15]:
## Calculate the average speed from the speed data and add it as a new column called 'AvgSpd'
df_vol_spd['AvgSpd'] = (df_vol_spd['MPH_0_15']*7.5+ df_vol_spd['MPH_15_25']*20+ 
                        df_vol_spd['MPH_25_30']*27.5+df_vol_spd['MPH_30_35']*32.5+ 
                        df_vol_spd['MPH_35_40']*37.5+df_vol_spd['MPH_40_45']*42.5+ 
                        df_vol_spd['MPH_45_50']*47.5+df_vol_spd['MPH_50_55']*52.5+ 
                        df_vol_spd['MPH_55_60']*57.5+df_vol_spd['MPH_60_65']*62.5+ 
                        df_vol_spd['MPH_65_70']*67.5+df_vol_spd['MPH_70_75']*72.5+ 
                        df_vol_spd['MPH_75_80']*77.5+df_vol_spd['MPH_80_85']*82.5+ 
                        df_vol_spd['MPH_85_UP']*85)/df_vol_spd['TotalVol']

## Add a new column called 'Source' and set the value to 'Speed Counter'
df_vol_spd['Source'] = "Speed Counter"

#### Data cleaning and Reasonableness check

In [16]:
# Filter out rows where AvgSpd is null and reset the DataFrame index
df_vol_spd = df_vol_spd[df_vol_spd['AvgSpd'].notnull()].reset_index(drop=True)

In [19]:
# Filter the DataFrame for rows where AvgSpd is 7.5, group them by LINKID, and then extract unique LINKIDs
unique_linkid_list = df_vol_spd.query('AvgSpd == 7.5').groupby('LINKID').size().index.tolist()

# Print the list of unique LINKIDs that meet the condition
print("Unique LINKIDs where average speed is 7.5 mph:", unique_linkid_list)


Unique LINKIDs where average speed is 7.5 mph: [10332, 10397, 10398, 10399, 10443, 10595, 20145, 20172, 20178, 20180, 20233, 20234, 20350, 40001, 40040, 40114, 40300, 40359, 40413, 40500, 40502, 40506, 50239, 50267, 60164, 60170, 80232, 80238, 80244, 80299, 80370, 80372, 80422, 80427, 80460, 80502, 90106, 90139, 90141, 90305, 110011, 110029, 120012, 120015, 120016, 120027, 120028, 140039, 140052, 140057, 140072, 140077, 140312, 150112, 150113, 150319, 180001, 180008, 180011, 180016, 180019, 180022, 180024, 180035, 190013, 190015, 199741, 199750, 199760, 781003, 781100, 781108, 781220, 781223, 781229, 781230, 781249, 781984, 781985, 786964, 787049, 787120, 787169, 787171, 789282, 789288, 789299, 789347, 789359, 789360, 789362, 789363, 789409, 789420, 789443, 789446, 789811, 789822, 790816, 792625, 795433, 880012, 880032, 920380, 940419, 980503]


In [21]:
df_vol_spd = df_vol_spd.query('AvgSpd > 7.5')

In [22]:
# Select specific columns from df_vol_spd and drop any duplicate rows based on those columns
df_vol_spd = df_vol_spd[["LINKID", "Date", "Hour", "Year", "Mon", "Day", "Wkday", "TotalVol", "AvgSpd", "Source"]].drop_duplicates()

In [23]:
# remove the LINKIDs that does not have full year  coverage
df_vol_spd = df_vol_spd[~df_vol_spd.LINKID.isin(list_coverage_missing)]

#### Filter out LINKIDs that have data but not enough entries

In [24]:
## Group the data by LINKID and count the number of rows for each group, saving the results to 'grouped'
grouped = df_vol_spd.groupby(['LINKID']).size().reset_index(name='counts')

## Set a filter value and filter 'grouped' to only include rows where 'counts' is greater than or equal to 'filter_value'
filter_value = 720
filtered_grouped = grouped[grouped['counts']>=filter_value]

## Extract the LINKIDs to keep based on 'filtered_grouped'
LINKID_to_keep = filtered_grouped.LINKID

## Filter the original DataFrame to only include rows where LINKID is in 'LINKID_to_keep'
filtered_df_vol_spd = df_vol_spd[df_vol_spd.LINKID.isin(LINKID_to_keep)]


In [25]:
print("Speed counter data that has less than 30 days coverage in a year")
df_vol_spd.query('Source =="Speed Counter"').groupby(["LINKID","Year"]).agg({'TotalVol':np.size}).reset_index().query('TotalVol<720')

Speed counter data that has less than 30 days coverage in a year


,LINKID,Year,TotalVol


In [26]:
print("filter: data entry more than", filter_value, "per LINKID")
print("number of LINKID before filter: ", len(df_vol_spd.LINKID.unique()))
print("number of LINKID after filter: ", len(filtered_df_vol_spd.LINKID.unique()))

unique_in_result =  set(df_vol_spd.LINKID.unique()) - set(filtered_df_vol_spd.LINKID.unique())

summary_df = pd.merge(summary_df, pd.DataFrame({'Not_enough_data_LINKID': list(unique_in_result)}),  how='left', left_on=['Requested_LINKID'], right_on =['Not_enough_data_LINKID'])
#summary_df

filter: data entry more than 720 per LINKID
number of LINKID before filter:  507
number of LINKID after filter:  507


In [26]:
df_vol_spd = filtered_df_vol_spd.copy()

In [27]:
import gc  # Importing the garbage collector module

# Delete individual DataFrames to free up memory
del filtered_df_vol_spd
del grouped
del filtered_grouped

# Call the garbage collector to free up memory
gc.collect()


84

In [28]:
print("Total LINKIDs from Speed counter:", len(df_vol_spd.query('Source == "Speed Counter"').LINKID.unique()))

Total LINKIDs from Speed counter: 507


### Add on data from volumn data and INRIX conflation

#### Volume distribution profile to approximate hourly volume and INRIX hourly speed

In [29]:
## Create distribution based on upstream linkid and apply to the AADT
## Group the data by LINKID, Year, and Hour, calculate the mean of TotalVol for each group, and save the result to 'df_AADT_dist'
df_AADT_dist = df_vol_spd.groupby(["LINKID","Year","Hour"]).agg({'TotalVol':np.mean}).reset_index()

## Rename the 'TotalVol' column to 'Hourly_Vol' in 'df_AADT_dist'
df_AADT_dist = df_AADT_dist.rename(columns={"TotalVol": "Hourly_Vol"})

## Group 'df_AADT_dist' by LINKID and Year, calculate the sum of Hourly_Vol for each group, and save the result to 'df_AADT_dist_sum'
df_AADT_dist_sum = df_AADT_dist.groupby(["LINKID","Year"]).agg({'Hourly_Vol':np.sum}).reset_index()

## Rename the 'Hourly_Vol' column to 'Total_Daily_Vol' in 'df_AADT_dist_sum'
df_AADT_dist_sum.rename(columns={"Hourly_Vol": "Total_Daily_Vol"}, inplace=True)

## Merge 'df_AADT_dist' with 'df_AADT_dist_sum' based on the 'LINKID' and 'Year' columns
df_AADT_dist = pd.merge(df_AADT_dist, df_AADT_dist_sum,  how='left', left_on=['LINKID','Year'], right_on =['LINKID','Year'])

## Create a new column in 'df_AADT_dist' called 'hourly_dist' and calculate the ratio of Hourly_Vol to Total_Daily_Vol for each row
df_AADT_dist['hourly_dist'] = df_AADT_dist['Hourly_Vol']/df_AADT_dist['Total_Daily_Vol'] 
df_AADT_dist = df_AADT_dist.rename(columns={"LINKID": "LINKID_to_ref"})

In [31]:
## obtain linkid ref from shapefile
LINKID_ref = (Seg_attr[['LINKID', 'Refer_LINKID', 'INRIX_XDID']]
              .dropna()
              .sort_values('LINKID')
              .rename(columns={"Refer_LINKID": "LINKID_ref", "INRIX_XDID": "XDSEG"})
              .reset_index(drop=True))

# Read the export_aadt.csv file into a DataFrame
LINKID_aadt_addon = pd.read_csv('test_tms_speed\\volume_counter_data\\ADT_distribution_volume\\export_aadt.csv')

In [32]:
# Print a message indicating the source and columns of LINKID_ref, and display the total number of unique LINKID with reference
print("LINKID_ref comes from the shapefile, selecting LINKID, Refer_LINKID, INRIX_XDID,\n    then remove duplicates, filter when Refer_LINKID is not null")
print(f"Total number of LINKID with reference: {len(LINKID_ref['LINKID'].unique())}")


LINKID_ref comes from the shapefile, selecting LINKID, Refer_LINKID, INRIX_XDID,
    then remove duplicates, filter when Refer_LINKID is not null
Total number of LINKID with reference: 779


#### Join with INRIX hourly Speed

In [33]:
# Convert the 'AADTYEAR' column to a datetime object and extract the year to a new 'Year' column
LINKID_aadt_addon['Year'] = pd.to_datetime(LINKID_aadt_addon['AADTYEAR'], infer_datetime_format=True).dt.year

# Merge LINKID_ref and LINKID_aadt_addon DataFrames based on the 'LINKID' column
df_dist_LINKID_ref = pd.merge(LINKID_ref, LINKID_aadt_addon, how='left', on=['LINKID'])

# Read the INRIX_spd_agg.csv file into a DataFrame
df_spd_agg = pd.read_csv('test_tms_speed\\volume_counter_data\\ADT_distribution_volume\\INRIX_data\\INRIX_spd_agg.csv')


In [39]:
# Merge df_dist_LINKID_ref and df_AADT_dist DataFrames based on 'LINKID_ref' and 'Year'
Dist_ADT = pd.merge(df_dist_LINKID_ref, df_AADT_dist, how='left', left_on=['LINKID_ref', 'Year'], right_on=['LINKID_to_ref', 'Year'])

# Calculate 'TotalVol' as the product of 'AADT' and 'hourly_dist'
Dist_ADT["TotalVol"] = Dist_ADT["AADT"] * Dist_ADT["hourly_dist"]

# Filter out rows where 'TotalVol' is null and reset the index
Dist_ADT = (Dist_ADT[Dist_ADT['TotalVol'].notnull()]
            .reset_index(drop=True))

# Merge Dist_ADT and df_spd_agg DataFrames based on 'XDSEG', 'Year', and 'Hour'
Dist_ADT = pd.merge(Dist_ADT, df_spd_agg, how='left', on=['XDSEG', 'Year', 'Hour'])
Dist_ADT = Dist_ADT.groupby(["LINKID","Year","Hour"]).agg({'TotalVol': 'mean','Spd': 'mean'}).reset_index()

In [40]:
# Create Pseudo date, day, month, and weekday columns and update data types
Dist_ADT = Dist_ADT.assign(
    Date=Dist_ADT['Year'] * 10000,
    Day=99.0,
    Mon=99.0,
    Wkday=99.0
).astype({
    'Date': np.int64,
    'TotalVol': np.int64
})
Dist_ADT = Dist_ADT.rename(columns={"hourly_Vol_dist": "TotalVol","Spd":"AvgSpd"})

In [41]:
print("Volume year-hour distribution using upstream reference LINKID")
print("due to count data availability, it is recommended to use data from 2017")
Dist_ADT.head()

Volume year-hour distribution using upstream reference LINKID
due to count data availability, it is recommended to use data from 2017


,LINKID,Year,Hour,TotalVol,AvgSpd,Date,Day,Mon,Wkday
0,10005.0,2017,0.0,402,63.320442,20170000,99.0,99.0,99.0
1,10005.0,2017,1.0,335,63.052055,20170000,99.0,99.0,99.0
2,10005.0,2017,2.0,295,62.975000,20170000,99.0,99.0,99.0
3,10005.0,2017,3.0,282,62.911846,20170000,99.0,99.0,99.0
4,10005.0,2017,4.0,324,62.891667,20170000,99.0,99.0,99.0


In [45]:
# Select specific columns, add a new 'Source' column, and count unique LINKIDs
Dist_ADT = (Dist_ADT[["LINKID", "Date", "Hour", "Year", "Mon", "Day", "Wkday", "TotalVol", "AvgSpd"]]
            .assign(Source="Vol Distribution&INRIX Spd"))
Dist_ADT['Source'] = "Vol Distribution&INRIX Spd"
print("Total LINKIDs from AADT distribution and INRIX:", len(Dist_ADT.LINKID.unique()))
print("total LINKIDs from speed counter", len(df_vol_spd.query('Source == "Speed Counter"').LINKID.unique()))
df_vol_spd = pd.concat([df_vol_spd,Dist_ADT], axis=0).reset_index()
del df_vol_spd['index']

Total LINKIDs from AADT distribution and INRIX: 779
total LINKIDs from speed counter 507


In [47]:
# Merge Study_area_LINKID and df_vol_spd based on 'LINKID'
# Filter out rows where 'TotalVol' is null and reset the index
df_vol_spd = pd.merge(Study_area_LINKID, df_vol_spd,  how='left', left_on=['LINKID'], right_on =['LINKID'])
df_vol_spd = df_vol_spd[df_vol_spd.TotalVol.notnull()].reset_index()
# Save the DataFrame to a CSV file without row numbers
df_vol_spd.to_csv('output\\vol_spd.csv', index=False)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

### Clean Road Attribute table and Join with df_vol_spd

In [50]:
# Rename columns, filter out rows, select specific columns, and drop duplicates
Seg_attr = (Seg_attr.rename(columns={
                "Lane _Capacity": "Lane_Capacity",
                "Tag_POST_SPD": "POST_SPD",
                "Lanes": "LANE_NUM"
            })
            .query('LINKID > 0')
            .loc[:, ["ID", "LINKID", "ROUTE_NAME", "Lane_Capacity", "LANE_NUM", "POST_SPD", "urban"]]
            .drop_duplicates())


In [51]:
# Extract the second to last character from the 'ROUTE_NAME' column and store it in a new 'Dir' column
Seg_attr['Dir'] = Seg_attr['ROUTE_NAME'].str[-2:-1]
# Calculate unique LINKIDs that are in Study_area_LINKID but not in Seg_attr
unique_in_request_list_from_attr = set(Study_area_LINKID.LINKID.unique()) - set(Seg_attr.LINKID.unique())

# Create a DataFrame from the list of missing LINKIDs and save it to a CSV file
(pd.DataFrame({'missing_data_LINKID': list(unique_in_request_list_from_attr)})
 .to_csv('output\\missing_attribute.csv', index=False))


In [53]:
# Display the first few rows of df_vol_spd DataFrame
df_vol_spd.head()

     LINKID        Date  Hour    Year   Mon   Day  Wkday  TotalVol     AvgSpd  \
0  940761.0  20170000.0   0.0  2017.0  99.0  99.0   99.0     226.0  66.578275   
1  940761.0  20170000.0   1.0  2017.0  99.0  99.0   99.0     171.0  66.415282   
2  940761.0  20170000.0   2.0  2017.0  99.0  99.0   99.0     147.0  66.052632   
3  940761.0  20170000.0   3.0  2017.0  99.0  99.0   99.0     151.0  66.101307   
4  940761.0  20170000.0   4.0  2017.0  99.0  99.0   99.0     214.0  66.236364   

                       Source  
0  Vol Distribution&INRIX Spd  
1  Vol Distribution&INRIX Spd  
2  Vol Distribution&INRIX Spd  
3  Vol Distribution&INRIX Spd  
4  Vol Distribution&INRIX Spd  


In [55]:
# Display the first few rows of Seg_attr DataFrame
Seg_attr.head()

,ID,LINKID,ROUTE_NAME,Lane_Capacity,LANE_NUM,POST_SPD,urban,Dir
0,1,940761.0,R-VA IS00295SB,2100,3,70,0,S
1,2,40046.0,R-VA IS00295SB,1600,4,70,1,S
2,3,50218.0,R-VA IS00264EB,1600,4,55,1,E
3,4,190077.0,R-VA IS00395SB,1600,2,55,1,S
4,5,50608.0,R-VA IS00264EB,1400,3,55,1,E


In [54]:
# Merge Seg_attr and df_vol_spd DataFrames based on 'LINKID' and drop rows with any NaN values
df_joined = (pd.merge(Seg_attr, df_vol_spd, how='left', on=['LINKID'])
             .dropna())


In [55]:
# Print the number of unique LINKIDs in df_vol_spd and df_joined
print(f"Number of LINKID with enough data: {len(df_vol_spd['LINKID'].unique())}")
print(f"Number of LINKID with enough data and Road Attributes: {len(df_joined['LINKID'].unique())}")

# Calculate unique LINKIDs that are in df_vol_spd but not in Seg_attr
unique_in_df_vol_spd = set(df_vol_spd['LINKID'].unique()) - set(Seg_attr['LINKID'].unique())

# Print the number of such unique LINKIDs
print(f"Number of LINKID with enough data but missing Road Attributes: {len(unique_in_df_vol_spd)}")

# Merge this information into summary_df
summary_df = pd.merge(
    summary_df,
    pd.DataFrame({'missing_attribute_LINKID': list(unique_in_df_vol_spd)}),
    how='left',
    left_on=['Requested_LINKID'],
    right_on=['missing_attribute_LINKID']
)

# Display the first few rows of the updated summary_df
summary_df.head()


Number of LINKID with enough data: 1232
Number of LINKID with enough data and Road Attributes: 1231
Number of LINKID with enough data but missing Road Attributes: 0


,Requested_LINKID,missing_data_LINKID,Not_enough_data_LINKID,missing_attribute_LINKID
0,90112.0,90112.0,NaN,NaN
1,90114.0,90114.0,NaN,NaN
2,90116.0,90116.0,NaN,NaN
3,90118.0,90118.0,NaN,NaN
4,90120.0,90120.0,NaN,NaN


### Calculate Free Flow Speed

In [56]:
# Current data structure
df_joined.head()

,ID,LINKID,ROUTE_NAME,Lane_Capacity,LANE_NUM,POST_SPD,urban,Dir,Date,Hour,Year,Mon,Day,Wkday,TotalVol,AvgSpd,Source
0,1,940761.0,R-VA IS00295SB,2100,3,70,0,S,20170000.0,0.0,2017.0,99.0,99.0,99.0,226.0,66.578275,Vol Distribution&INRIX Spd
1,1,940761.0,R-VA IS00295SB,2100,3,70,0,S,20170000.0,1.0,2017.0,99.0,99.0,99.0,171.0,66.415282,Vol Distribution&INRIX Spd
2,1,940761.0,R-VA IS00295SB,2100,3,70,0,S,20170000.0,2.0,2017.0,99.0,99.0,99.0,147.0,66.052632,Vol Distribution&INRIX Spd
3,1,940761.0,R-VA IS00295SB,2100,3,70,0,S,20170000.0,3.0,2017.0,99.0,99.0,99.0,151.0,66.101307,Vol Distribution&INRIX Spd
4,1,940761.0,R-VA IS00295SB,2100,3,70,0,S,20170000.0,4.0,2017.0,99.0,99.0,99.0,214.0,66.236364,Vol Distribution&INRIX Spd


In [57]:
## Create an empty DataFrame called 'df_FFSPD'
df_FFSPD = pd.DataFrame()

## Group the 'df_joined' DataFrame by 'LINKID' and filter for rows where 'AvgSpd' is greater than 'POST_SPD'
grouped = df_joined.query('AvgSpd > POST_SPD').groupby('LINKID')

## For each unique value of 'LINKID', calculate the average of the 'AvgSpd' column and create a new column called 'FFSpd' in the group DataFrame for speeds above 'POST_SPD'
## Also create a new column called 'FF_points' in the group DataFrame for the number of points above 'POST_SPD'
## Then, create a new DataFrame called 'new_row' that contains the LINKID, FFSpd, and FF_points for the group
## Append 'new_row' to 'df_FFSPD' and reset the index of the resulting DataFrame
for name, group in grouped:
    mask = (group['TotalVol'] / group['LANE_NUM']) < 500
    group['FFSpd'] = group['AvgSpd'][mask].mean()
    group['FF_points'] = group['AvgSpd'][mask].count()
    
    new_row = pd.DataFrame({'LINKID': name, 'FFSpd':group['FFSpd'].mean(), 'FF_points':group['FF_points'].mean()}, index=[0])
    df_FFSPD = pd.concat([new_row,df_FFSPD.loc[:]]).reset_index(drop=True)

## Print the first 10 rows of the resulting DataFrame
df_FFSPD.head(10)


,LINKID,FFSpd,FF_points
0,990120.0,62.817330,32.0
1,990112.0,58.094294,33.0
2,990108.0,58.771397,46.0
3,980503.0,71.928605,17190.0
4,980463.0,70.771581,13.0
5,980300.0,71.492136,8627.0
6,980296.0,67.107190,56.0
7,980254.0,70.635326,13.0
8,980188.0,72.076891,52.0
9,980011.0,71.022799,10.0


In [58]:
## Merge the 'df_joined' and 'df_FFSPD' DataFrames based on the 'LINKID' column and save the result to a new DataFrame called 'df_joined_FFSpd'
df_joined_FFSpd = pd.merge(df_joined, df_FFSPD,  how='left', left_on=['LINKID'], right_on =['LINKID'])

## Replace missing values in the 'FFSpd' column with the value of 'POST_SPD' + 7 where the 'FFSpd' column is null
df_joined_FFSpd.loc[df_joined_FFSpd['FFSpd'].isnull(),'FFSpd'] = df_joined_FFSpd['POST_SPD'] + 7

## Replace missing values in the 'FF_points' column with 0 where the 'FFSpd' column is null
df_joined_FFSpd.loc[df_joined_FFSpd['FFSpd'].isnull(),'FF_points'] = 0


In [59]:
# Delete the DataFrames to free up memory
del grouped
del df_joined

# Call the garbage collector to free up memory
import gc
gc.collect()


13

### Area type lookup

In [62]:
# Rename the 'urban' column to 'Area_type'
df_joined_FFSpd.rename(columns={"urban": "Area_type"}, inplace=True)

# Print the number of unique LINKIDs in df_joined_FFSpd
print("number of LINKID with Area Type info: ", len(df_joined_FFSpd.LINKID.unique()))

# Print the number of such unique LINKIDs
unique_in_df_joined_FFSpd = set(df_joined_FFSpd.query('Area_type!=Area_type').LINKID.unique()) - set(df_joined_FFSpd.LINKID.unique())

print("number of LINKID with enough data but missing Area Type: ", len(list(unique_in_df_joined_FFSpd)))

# Merge this information into summary_df
summary_df = pd.merge(summary_df, pd.DataFrame({'missing_Area_Type_LINKID': list(unique_in_df_joined_FFSpd)}),  how='left', left_on=['Requested_LINKID'], right_on =['missing_Area_Type_LINKID'])
summary_df.head()

number of LINKID with Area Type info:  1231
number of LINKID with enough data but missing Area Type:  0


,Requested_LINKID,missing_data_LINKID,Not_enough_data_LINKID,missing_attribute_LINKID,missing_Area_Type_LINKID_x,missing_Area_Type_LINKID_y,missing_Area_Type_LINKID
0,90112.0,90112.0,NaN,NaN,NaN,NaN,NaN
1,90114.0,90114.0,NaN,NaN,NaN,NaN,NaN
2,90116.0,90116.0,NaN,NaN,NaN,NaN,NaN
3,90118.0,90118.0,NaN,NaN,NaN,NaN,NaN
4,90120.0,90120.0,NaN,NaN,NaN,NaN,NaN


In [63]:
# Calculate Density based on Volume and Lane number
df_joined_FFSpd['Density'] = (df_joined_FFSpd['TotalVol']/df_joined_FFSpd['LANE_NUM'])/df_joined_FFSpd['AvgSpd']

### Lookup LOS result based on available density and area type

In [64]:
## These two lines filter out rows where the value in the "Area_type" column is missing (NaN).
df_joined_FFSpd = df_joined_FFSpd.query('Area_type==Area_type')

## These two lines filter out rows where the value in the "AvgSpd" column is missing (NaN).
df_joined_FFSpd = df_joined_FFSpd.query('AvgSpd==AvgSpd')

In [65]:
## input shapefile after Peng's lane number update on March 5th, 
## contains 9 records Area type not 0 or 1 
## currently assigning as 1 since they are all 1
df_joined_FFSpd.loc[df_joined_FFSpd['Area_type'] > 1, 'Area_type'] = 1


In [66]:
import numpy as np

# Define the lookup tables as numpy arrays
lookup_area_type_0 = np.array([[11,  'A'],
                               [18, 'B'],
                               [26, 'C'],
                               [35, 'D'],
                               [45,  'E'],
                               [999, 'F']])
lookup_area_type_1 = np.array([[6, 'A'],
                               [14, 'B'],
                               [22, 'C'],
                               [29, 'D'],
                               [39, 'E'],
                               [999, 'F']])

# Map area type values to lookup arrays
lookup = {0: lookup_area_type_0, 1: lookup_area_type_1}

def get_los(density, area_type):
    # Get the lookup array for the given area type
    look = lookup[area_type]
    # Use numpy's `searchsorted` method to find the index of the first item in the array
    # that is greater than or equal to the density
    idx = np.searchsorted(look[:, 0].astype(float), density)
    
    # Get the LOS value based on the index
    if idx< len(look):
        los = look[idx, 1] 
    else: 
        los = look[idx-1, 1] 
    
    return los


df_joined_FFSpd['LOS'] = df_joined_FFSpd.apply(lambda row: get_los(row['Density'], row['Area_type']), axis=1)

In [72]:
df_joined_FFSpd.to_csv("output\\output_With_spd.csv")

In [73]:
summary_df.to_csv('output\\Current_Available_LINKID.csv')

In [82]:
print("examples for LOS calculation result")
df_joined_FFSpd.query('LINKID == 40766 and Year == 2021 and Hour == 7').head()

,ID,LINKID,ROUTE_NAME,Lane_Capacity,LANE_NUM,POST_SPD,Area_type,Dir,Date,Hour,Year,Mon,Day,Wkday,TotalVol,AvgSpd,Source,FFSpd,FF_points,Density,LOS
10432448,697,40766.0,R-VA IS00064EB,1800,3,60,1,E,20210101.0,7.0,2021.0,1.0,1.0,4.0,534.0,64.794007,Speed Counter,62.874995,15879.0,2.747168,A
10432472,697,40766.0,R-VA IS00064EB,1800,3,60,1,E,20210102.0,7.0,2021.0,1.0,2.0,5.0,810.0,65.246914,Speed Counter,62.874995,15879.0,4.138127,A
10432496,697,40766.0,R-VA IS00064EB,1800,3,60,1,E,20210103.0,7.0,2021.0,1.0,3.0,6.0,560.0,63.473214,Speed Counter,62.874995,15879.0,2.940873,A
10432520,697,40766.0,R-VA IS00064EB,1800,3,60,1,E,20210104.0,7.0,2021.0,1.0,4.0,0.0,3422.0,64.544126,Speed Counter,62.874995,15879.0,17.672664,C
10432544,697,40766.0,R-VA IS00064EB,1800,3,60,1,E,20210105.0,7.0,2021.0,1.0,5.0,1.0,3552.0,64.455236,Speed Counter,62.874995,15879.0,18.369338,C


#### LOS result reasonableness check

In [80]:
print("Maybe double check these LINKID LANE NUMBER since the speed is above 60 and LOS is at E or F from lookup table")
df_joined_FFSpd[['ID','LINKID', 'Year', 'Hour', 'AvgSpd', 'Density', 'TotalVol', 'LANE_NUM', 'Dir','LOS']].query('AvgSpd > 60 and LOS in ["E","F"]').LINKID.unique()

Maybe double check these LINKID LANE NUMBER since the speed is above 60 and LOS is at E or F from lookup table


array([190077., 190073., 190075., 150318., 140056., 190078.,  50217.,
       190074., 190071., 190174., 190066., 190072., 150066., 150069.,
       840083., 140082., 140081., 150067., 140080., 150064., 150070.,
       150051., 150065., 794135., 794132., 190012., 190013., 140236.,
       190188., 140017., 190081., 140020., 140018., 160001., 140006.,
       140019., 160004., 140012., 140243.,  50307.,  40777.,  50306.,
        40108.,  50308.,  50155.,  40766.,  50305.,  50165.,  50169.,
       150013.,  50598., 840070., 240500., 150041., 150024., 140051.,
       794136., 199760., 150112.,  50204.,  50200., 150113., 190038.,
       190032., 190035., 190030., 190040., 190041.,  20475.,  20176.,
        10397.,  80373.,  90305.,  90206.,  40491., 140003.,  90204.,
        50117.,  90200.,  40300.,  40298.,  40361.,  50587.,  40360.,
        40304.,  60315., 190001., 190306., 190068., 190067.,  90272.,
        90114.,  90112.,  90106.,  90118.,  90110.,  90116., 795420.,
       789485., 7954